In [10]:
import os
import SimpleITK as sitk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import argparse
from NetworkTrainer.network_trainer import *
import torch
from Evaluate.evaluate import *
#from SVVNet.IVDSegmentation.test import copy_sitk_imageinfo
from utils.processing import resize_sitk_image
from utils.tools import csv_to_catalogue

#csv_path = '../../Catalogue' + '/' + str(args.catalogue) + '.csv'
#csv_path = 'Catalogue' + '/' + '0.csv'
#catalogue = csv_to_catalogue(csv_path)
#cases = catalogue['test'].dropna()
list_case_dirs = dict()
cases = os.listdir('/home/minghao/Desktop/Output/Vertebrae_Segmentation/Prediction')
cases

['Case116',
 'Case134',
 'Case133',
 'Case130',
 'Case144',
 'Case110',
 'Case145',
 'Case178',
 'Case16',
 'Case177',
 'Case79',
 'Case198',
 'Case78',
 'Case158',
 'Case6',
 'Case215',
 'Case212',
 'Case188',
 'Case199',
 'Case81',
 'Case48',
 'Case154']

In [11]:
data_path = '/home/minghao/Desktop/Spine_Segmentation'
save_path = '/home/minghao/Desktop/Output/Joint'

if not os.path.exists(save_path):
    os.mkdir(save_path)

In [12]:
Coccyx_path = '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction'
list_case_dirs['Coccyx'] = [os.path.join(Coccyx_path, cases[i]) for i in range(len(cases))]

In [13]:
list_case_dirs['Coccyx']

['/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case116',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case134',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case133',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case130',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case144',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case110',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case145',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case178',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case16',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case177',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case79',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case198',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case78',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case1

In [14]:
IVD_path = '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction'
list_case_dirs['IVD'] = [os.path.join(IVD_path, cases[i]) for i in range(len(cases))]

In [15]:
list_case_dirs['IVD']

['/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case116',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case134',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case133',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case130',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case144',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case110',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case145',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case178',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case16',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case177',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case79',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case198',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case78',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case158',
 '/home/minghao/Desktop/Output/IVD_Se

In [16]:
Vertebrae_path = '/home/minghao/Desktop/Output/Vertebrae_Segmentation/Prediction'
list_case_dirs['Vertebrae'] = [os.path.join(Vertebrae_path, cases[i]) for i in range(len(cases))]


In [17]:
def copy_sitk_imageinfo(image1, image2):
    image2.SetSpacing(image1.GetSpacing())
    image2.SetDirection(image1.GetDirection())
    image2.SetOrigin(image1.GetOrigin())

    return image2

def landmark_extractor(landmarks):
    """
    return a list of the landmarks
    :param landmarks: pandas.Dataframe
    """
    labels = landmarks.columns[1:].tolist()  # exclude the 'axis' column
    list_landmarks = []
    for label in labels:
        list_landmarks.append(np.array(landmarks[label]))

    return list_landmarks

In [18]:
dict_images = dict()
lable1_array = []
lable2_array = []
intersection = []
files_name = ['Coccyx', 'IVD','Vertebrae']
joint = []
S = (12,512,512)

for num in  range(len(cases)):
    Joint_nii = np.zeros(S)

    pre_Coccyx = sitk.ReadImage(list_case_dirs[files_name[0]][num] + '/' + 'pred_' + files_name[0] + 'Mask.nii.gz', sitk.sitkFloat32)
    pre_Coccyx = sitk.GetArrayFromImage(pre_Coccyx)

    pre_IVD = sitk.ReadImage(list_case_dirs[files_name[1]][num] + '/' + 'pred_' + files_name[1] + 'Mask.nii.gz', sitk.sitkFloat32)
    pre_IVD = sitk.GetArrayFromImage(pre_IVD)

    pre_Vertebrae = sitk.ReadImage(list_case_dirs[files_name[2]][num] + '/' + 'pred_' + files_name[2] + 'Mask.nii.gz', sitk.sitkFloat32)
    pre_Vertebrae = sitk.GetArrayFromImage(pre_Vertebrae)

    landmark = pd.read_csv(data_path + '/'+ cases[num] + '/landmarks_512.csv')
    landmarks = landmark_extractor(landmark)

    temp = pre_IVD + pre_Vertebrae

    lable_2 = np.where(temp == 2)
    for i in range(len(lable_2[0])):
        array = []
        array.append(lable_2[0][i])
        array.append(lable_2[1][i])
        array.append(lable_2[2][i])
        lable2_array.append(array)

    lable_1 = np.where(pre_Coccyx == 1)
    for i in range(len(lable_1[0])):
        array = []
        array.append(lable_1[0][i])
        array.append(lable_1[1][i])
        array.append(lable_1[2][i])
        lable1_array.append(array)
        
    for i in range(len(lable1_array)):
        if lable1_array[i] in lable2_array:
            intersection.append(lable1_array[i])


    Joint_nii = pre_Coccyx + temp
    for i in range(len(intersection)):
        np.put(Joint_nii,intersection[i],2)


    template_nii = sitk.ReadImage(data_path + '/' + cases[num] + '/MR_512.nii.gz')
    Joint_nii = sitk.GetImageFromArray(Joint_nii)
    Joint_nii = copy_sitk_imageinfo(template_nii, Joint_nii)
    if not os.path.exists(save_path + '/' + cases[num]):
            os.mkdir(save_path + '/' + cases[num])
    sitk.WriteImage(Joint_nii, save_path + '/' + cases[num] + '/pred_JointMask.nii.gz')
print('Done')

KeyboardInterrupt: 

In [ ]:
Joint_path = '/home/minghao/Desktop/Output/Joint'

list_case_dirs['Joint'] = [os.path.join(Joint_path, cases[i]) for i in range(len(cases))]
dscs = []
for case_dir in range(len(list_case_dirs['Joint'])):

    gt_mask = sitk.ReadImage(data_path + '/' + cases[case_dir] + '/raw_Mask.nii.gz')
    gt = sitk.GetArrayFromImage(gt_mask)
    D, H, W = gt.shape

    pred_mask = sitk.ReadImage(list_case_dirs['Joint'][case_dir] + '/' + 'pred_JointMask.nii.gz')
    pred = resize_sitk_image(pred_mask, dsize=(W, H, None), interpolator='nearest')
    pred = sitk.GetArrayFromImage(pred)

    dsc = cal_subject_level_dice(pred, gt, num_classes=20)
    dscs.append(dsc)

In [ ]:
np.mean(dscs)
